<a href="https://colab.research.google.com/github/DrBryne/colab/blob/main/Vertex_AI_Grounding_Web_and_Vertex_AI_Search_Datastores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Grounding for PaLM LLM

This notebook explores the Python libraries for Vertex AI Grounding.


| | |
|----------|-------------|
| Author(s)   | Mandie Quartly (mandieq@), Victor Dantas (vdantas@) |
| Last update | 2023-12-06 Added grounding on Vertex AI Search datastore |

In [1]:
!pip install google-cloud-aiplatform --upgrade

In [2]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [23]:
project_id = "osp-bu-desktop-skynet" # @param
location = "us-central1" # @param
datastore_id = "DATASTORE ID" # @param
datastore_location = "global" # @param

In [24]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project=project_id, location=location)

See SDK details for [predict](https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.TextGenerationModel#vertexai_language_models_TextGenerationModel_predict).

## Grounding Responses with Web Search

In [25]:
web = vertexai.language_models._language_models.WebSearch()

In [26]:
parameters = {
    "candidate_count": 1,
    "grounding_source": web, # note not grounding_config per the code given in the UI
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_p": 0.8,
    "top_k": 40
}

model = TextGenerationModel.from_pretrained("text-bison")

In [27]:
try:
  response = model.predict(
      """Who is the CEO of Opera?""",
      **parameters
  )
  print(f"Response from Model: \n{response.text}")
  print(f"\nCitations: \n{response.grounding_metadata.citations}")
except Exception as exc:
  print("ERROR:", exc.message)



ERROR: Permission 'aiplatform.endpoints.predict' denied on resource '//aiplatform.googleapis.com/projects/osp-bu-desktop-skynet/locations/us-central1/publishers/google/models/text-bison@002' (or it may not exist).


In [17]:
import json
print(json.dumps(response.raw_prediction_response, indent=2))

[
  [
    {
      "safetyAttributes": {
        "categories": [
          "Derogatory",
          "Finance",
          "Health",
          "Insult",
          "Religion & Belief",
          "Sexual"
        ],
        "blocked": false,
        "scores": [
          0.1,
          0.1,
          0.2,
          0.1,
          0.3,
          0.1
        ],
        "safetyRatings": [
          {
            "severityScore": 0.0,
            "severity": "NEGLIGIBLE",
            "category": "Dangerous Content",
            "probabilityScore": 0.1
          },
          {
            "severityScore": 0.1,
            "severity": "NEGLIGIBLE",
            "probabilityScore": 0.1,
            "category": "Harassment"
          },
          {
            "probabilityScore": 0.1,
            "category": "Hate Speech",
            "severityScore": 0.1,
            "severity": "NEGLIGIBLE"
          },
          {
            "probabilityScore": 0.1,
            "category": "Sexually Explicit",
  

## Grounding Responses with a Vertex AI Search Datastore

In [ ]:
query = "how does Google make money?" # @param

In [ ]:
searchclass = vertexai.language_models.GroundingSource.VertexAISearch(
    data_store_id=datastore_id,location=datastore_location)

parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0,
    "top_p": 0.8,
    "top_k": 40,
    "grounding_source":searchclass,
}

response = model.predict(query,
    **parameters
)

src_document = None

try:
    src_document = response.grounding_metadata.citations[0].url.split('/')[-1]
except Exception as e:
    print("Could not fetch source document:", e)

print(response.text)
if src_document:
    print("\nSource:", src_document)

 Google generates revenue through various sources, including:

1. Google Ads: Google's primary source of revenue comes from advertising, particularly through its search engine and other advertising platforms. Businesses pay Google to display their ads on search results pages and other websites, and Google charges based on clicks or impressions.

2. Google Cloud: Google Cloud offers a range of cloud computing services, including infrastructure, platform, and software solutions, to businesses and organizations. Customers pay subscription fees or usage-based charges for these services.

3. Google Workspace: Google Workspace provides cloud-based collaboration and productivity tools, such as Gmail, Docs, Drive, Calendar, and Meet, to businesses and organizations. Customers pay subscription fees for access to these tools.

4. Google Play: Google Play is an app store where users can purchase and download apps, games, movies, books, and other digital content. Google takes a commission on each 

In [ ]:
response.raw_prediction_response

Prediction(predictions=[{'groundingMetadata': {'searchQueries': ['How does Google make money?'], 'citations': [{'endIndex': 62.0, 'url': 'gs://vdantas_cloud-llm-preview3/alphabet-public-docs/Alphabet 10k Annual Report 2022.pdf', 'startIndex': 1.0}]}, 'content': " Google generates revenue through various sources, including:\n\n1. Google Ads: Google's primary source of revenue comes from advertising, particularly through its search engine and other advertising platforms. Businesses pay Google to display their ads on search results pages and other websites, and Google charges based on clicks or impressions.\n\n2. Google Cloud: Google Cloud offers a range of cloud computing services, including infrastructure, platform, and software solutions, to businesses and organizations. Customers pay subscription fees or usage-based charges for these services.\n\n3. Google Workspace: Google Workspace provides cloud-based collaboration and productivity tools, such as Gmail, Docs, Drive, Calendar, and M